# Various Routines to Harvest CRIM Metadata from Production Server

### Load the data from CRIM

In [9]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize
import os.path

In [10]:



# FOR PRODUCTION DJANGO
# rel_data = requests.get('http://crimproject.org/data/relationships/').json()
# obs_data = requests.get('https://crimproject.org/data/observations/').json()
people = requests.get('https://crimproject.org/data/people/').json()
pieces = requests.get('https://crimproject.org/data/pieces/').json()
voices = requests.get('https://crimproject.org/data/voices/').json()
masses = requests.get('https://crimproject.org/data/masses/').json()
# phrases = requests.get('https://crimproject.org/data/phrases/').json()
roles = requests.get('https://crimproject.org/data/roles/').json()

# For Local DJANGO

# people = requests.get('http://127.0.0.1:8000/data/people/').json()
# pieces = requests.get('http://127.0.0.1:8000/data/pieces/').json()
# voices = requests.get('http://127.0.0.1:8000/data/voices/').json()
# masses = requests.get('http://127.0.0.1:8000/data/masses/').json()
# # phrases = requests.get('http://127.0.0.1:8000/data/phrases/').json()
# roles = requests.get('http://127.0.0.1:8000/data/roles/').json()

In [3]:
# people

### First Masses

In [4]:
# This is how the CRIM Django JSON looks for Masses:
sample = people[0]
sample

{'url': 'https://crimproject.org/data/people/CRIM_Person_0001/',
 'person_id': 'CRIM_Person_0001',
 'name': 'Pierre Colin',
 'name_sort': 'Colin, Pierre',
 'name_alternate_list': '',
 'birth_date': '15xx',
 'death_date': '15xx',
 'active_date': '1538–1572',
 'role_types': [{'url': 'https://crimproject.org/data/roletypes/composer/',
   'role_type_id': 'composer',
   'name': 'Composer'}]}

In [11]:
# DF based on that JSON
masses_from_json = pd.json_normalize(masses)
# masses_from_json

### New Heading

- This is our `notebook`

In [13]:
mass_list = masses_from_json['mass_id'].to_list()
# mass_list

In [6]:
# Here we load the newly-edited CSV of Mass metadata
# masses_data = pd.read_csv('CRIM_Mass_Django_Data.csv') 

mass_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQJh5BnhhOMGsGko7oGXCpLBqL4uLoetx2NlIvWL1SPPZRITOERaUpaPB23X7N70DrIF7q9_O58eout/pub?output=csv'
masses_data = pd.read_csv(mass_csv)
masses_data.rename(columns={"url": "crim_url"}, inplace=True)
# masses_data

In [7]:
# masses_data['mass_id'].to_list()

In [8]:
new_masses = masses_data[~masses_data['mass_id'].isin(mass_list)]
sample = new_masses.iloc[0]
# new_masses.head()

KeyError: 'mass_id'

In [19]:
def get_mass_metadata_from_df(sample):
    
    mass_id = sample['crim_id']
    title = sample['title']
    remarks = ''
    mass_data = {"model": "crim.crimmass",       
         'fields': {'mass_id': mass_id,
         'title': title,
         'genre': 'mass',
         'remarks': remarks}}
    return mass_data

In [25]:
# sample transformation of one mass

get_mass_metadata_from_df(sample)

{'model': 'crim.crimmass',
 'fields': {'mass_id': nan, 'title': 'Kyrie', 'genre': 'mass', 'remarks': ''}}

In [29]:
# run for all rows

masses_json = masses_data.apply(get_mass_metadata_from_df, axis=1).to_list()
# masses_json

In [21]:
## all steps at once

# masses = requests.get('http://127.0.0.1:8000/data/masses/').json() # pulls JSON from CRIM

masses_df = pd.json_normalize(masses) # JSON as dataframe

mass_list = masses_df['mass_id'].to_list() # get the ids of pieces already in CRIM as list

masses_data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQiEPbv0G3t3zAD9BmtrXGomZlCUdMRJitX6cNoQ7YYJzn1DicsymCP6WOJHiJBga3c0XVHq5lp3F8z/pub?output=csv') # get the full table from Freedman's Gsheet

new_masses = masses_data[~masses_data['crim_id'].isin(mass_list)] # keep ONLY Freedman items NOT in the CRIM Data
# 

new_masses_json = new_masses.apply(get_mass_metadata_from_df, axis=1).to_list() # apply the json building function 

with open("new_masses.json", "w") as write_file:
    json.dump(new_masses_json, write_file, indent=4) # writes the nice JSON file

,url,crim_id,title,comp_name,comp_id,comp_url,date,sort_date,min_voices,max_voices,remarks


### Now pieces

In [ ]:
pieces_from_json = pd.json_normalize(pieces)


In [ ]:
# CRIM_Piece_Django_Data.csv

piece_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQzG1TYOBUmbwMdRj8hf1h1UWfKjnTmAsy5c9Q4b5UftortFaKJPxJnRXphwH8HU9tLkkfQWUBJW781/pub?output=csv'

pieces_data = pd.read_csv(piece_csv) 
pieces_data['genre_title'].str.lower()

# masses_data.rename(columns={"url": "crim_url"}, inplace=True)
pieces_data.columns

In [ ]:
sample = pieces_data.iloc[280]
sample

In [ ]:
def get_piece_metadata_from_df(sample):
    piece_id = sample['crim_id']
    mass_id = sample['crim_mass_id']
    title = sample['title']
    genre = sample['genre_title']
    pdf = sample['pdf_link']
    mei = sample['mei_link']
    remarks = ''
    if mass_id == 'none':
        piece_data = {"model": "crim.crimpiece",       
         'fields': {'piece_id': piece_id,
         'title': title,
        'genre': genre,
        'mei_links': mei,
        'pdf_links': pdf,    
         'remarks': remarks}}
    else:
        piece_data = {"model": "crim.crimpiece",       
         'fields': {'piece_id': piece_id,
         'title': title,
        'genre': genre,
        'mei_links': mei,
        'pdf_links': pdf,
        'mass': mass_id,      
         'remarks': remarks}}
    return piece_data

In [ ]:
get_piece_metadata_from_df(sample)

In [ ]:
piece_list = pieces_from_json['piece_id'].to_list()
new_pieces = pieces_data[~pieces_data['crim_id'].isin(piece_list)]
sample = new_pieces.iloc[0]

In [ ]:
pieces_json = pieces_data.apply(get_piece_metadata_from_df, axis=1).to_list()

In [ ]:
with open("pieces.json", "w") as write_file:
    json.dump(pieces_json, write_file, indent=4)

In [ ]:
# All Steps at once
pieces = requests.get('https://crimproject.org/data/pieces/').json()

pieces_from_json = pd.json_normalize(pieces)
piece_list = pieces_from_json['piece_id'].to_list()

piece_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQzG1TYOBUmbwMdRj8hf1h1UWfKjnTmAsy5c9Q4b5UftortFaKJPxJnRXphwH8HU9tLkkfQWUBJW781/pub?output=csv'

pieces_data = pd.read_csv(piece_csv) 
pieces_data['genre_title'].str.lower()

new_pieces = pieces_data[~pieces_data['crim_id'].isin(piece_list)]

new_pieces_json = new_pieces.apply(get_piece_metadata_from_df, axis=1).to_list()

with open("new_pieces.json", "w") as write_file:
    json.dump(new_pieces_json, write_file, indent=4)

### Now Voices

In [5]:
# voices = requests.get('http://127.0.0.1:8000/data/voices/').json()


voices_from_json = pd.json_normalize(voices)
voices_from_json.head(3)

,url,order,original_name,regularized_name,clef,remarks,piece.url,piece.title
0,https://crimproject.org/data/voices/CRIM_Mass_...,1,Superius,Superius,,,https://crimproject.org/data/pieces/CRIM_Mass_...,Kyrie
1,https://crimproject.org/data/voices/CRIM_Mass_...,2,Contratenor,Contratenor,,,https://crimproject.org/data/pieces/CRIM_Mass_...,Kyrie
2,https://crimproject.org/data/voices/CRIM_Mass_...,3,Tenor,Tenor,,,https://crimproject.org/data/pieces/CRIM_Mass_...,Kyrie


In [ ]:
voices_from_json.columns


In [6]:

# this is for local
# voices_from_json['nurl'] = voices_from_json["url"].str.replace("http://127.0.0.1:8000/data/voices/","")

# this for production
voices_from_json['nurl'] = voices_from_json["url"].str.replace("https://crimproject.org/data/voices/","")

voice_list = voices_from_json['nurl'].str[:-1].to_list()
# voice_list

/var/folders/_s/4t2p1z2x0yxcv068dtqj31tw0000gq/T/ipykernel_57877/517080671.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  voices_from_json['nurl'] = voices_from_json["url"].str.replace("https://crimproject.org/data/voices/","")


In [7]:
voices_data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQiJKCXjLtTCdXZms94NFBywrD1r4fyJFNFZmmj54W7rlfI6ubf826EorBbGe6qUPOe9uutnGUdQiNa/pub?output=csv')
voices_data.tail(5)

,piece_id,voice_id,Unnamed: 2,Unnamed: 3,order,original_name,in_edition,regularized_name,clef,remarks,piece.url,piece.title
1318,CRIM_Model_0050,CRIM_Model_0050(2),(,),2,Sextus,Sextus,Sextus,NaN,NaN,NaN,NaN
1319,CRIM_Model_0050,CRIM_Model_0050(3),(,),3,Altus,Altus,Altus,NaN,NaN,NaN,NaN
1320,CRIM_Model_0050,CRIM_Model_0050(4),(,),4,Tenor,Tenor,Tenor,NaN,NaN,NaN,NaN
1321,CRIM_Model_0050,CRIM_Model_0050(5),(,),5,Quintus,Quintus,Quintus,NaN,NaN,NaN,NaN
1322,CRIM_Model_0050,CRIM_Model_0050(6),(,),6,Bassus,Bassus,Bassus,NaN,NaN,NaN,NaN


In [ ]:
voices_data['voice_id'].unique

In [17]:
def get_voices_metadata_from_df(sample):
    piece_id = sample['piece_id']
    voice_id = sample['voice_id']
    order = sample['order']
    orig_name = sample['original_name']
    reg_name = sample['regularized_name']
    remarks = ''
    if orig_name == 'NaN':
        voice_data = {'model': 'crim.crimvoice',
            'fields': {'piece_id': piece_id,
             'voice_id': voice_id,
            'order': order,
            'original_name': orig_name,
            'regularized_name': reg_name,
             'remarks': remarks}}
    else:
        voice_data = {'model': 'crim.crimvoice',
            'fields': {'piece_id': piece_id,
             'voice_id': voice_id,
            'order': order,
            'original_name': '',
            'regularized_name': reg_name,
             'remarks': remarks}}
    return voice_data

In [61]:
sample = voices_data.iloc[969]
get_voices_metadata_from_df(sample)

{'model': 'crim.crimvoice',
 'fields': {'piece_id': 'CRIM_Mass_0044_4',
  'voice_id': 'CRIM_Mass_0044_4(2)',
  'order': 2,
  'original_name': '',
  'regularized_name': 'Cantus 2',
  'remarks': ''}}

In [ ]:
new_voices = voices_data[~voices_data['voice_id'].isin(voice_list)]
# nv = new_voices.head(10)
# nv

In [ ]:
new_voices_json = new_voices.apply(get_voices_metadata_from_df, axis=1).to_list()
# new_voices_json

In [ ]:
with open("new_voices.json", "w") as write_file:
    json.dump(new_voices_json, write_file, indent=4)

In [ ]:
# tests

a = voices_data.groupby('piece_id')['order'].value_counts()
b = voices_data.groupby('piece_id')['regularized_name'].value_counts()
print(b)


In [18]:
# all steps at once

voices = requests.get('https://crimproject.org/data/voices/').json()

voices_from_json = pd.json_normalize(voices)

voices_from_json['nurl'] = voices_from_json["url"].str.replace("https://crimproject.org/data/voices/","")

voice_list = voices_from_json['nurl'].str[:-1].to_list()

voices_data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQiJKCXjLtTCdXZms94NFBywrD1r4fyJFNFZmmj54W7rlfI6ubf826EorBbGe6qUPOe9uutnGUdQiNa/pub?output=csv')

new_voices = voices_data[~voices_data['voice_id'].isin(voice_list)]

new_voices_json = new_voices.apply(get_voices_metadata_from_df, axis=1).to_list()

with open("new_voices.json", "w") as write_file:
    json.dump(new_voices_json, write_file, indent=4)

/var/folders/_s/4t2p1z2x0yxcv068dtqj31tw0000gq/T/ipykernel_58198/2984708587.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  voices_from_json['nurl'] = voices_from_json["url"].str.replace("https://crimproject.org/data/voices/","")


AttributeError: 'DataFrame' object has no attribute 'to_list'

### Roles


In [51]:
roles_from_json = pd.json_normalize(roles)
roles_from_json.columns


Index(['url', 'person', 'role_type', 'date', 'mass', 'piece', 'treatise',
       'source', 'remarks'],
      dtype='object')

In [91]:
role_data_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRcv1DLLiUZ3e9hIvYL6c3bW6ELJ5OJHlEVsqZvk9yKaARj4mR-onmGMBo8vGo4L9nLzA_8YpjuPNhq/pub?output=csv'
roles_data = pd.read_csv(role_data_csv) 
roles_data.head(10)

,Unnamed: 0,crim_id,mass_id,title,full_title,person_id,person,date,role_type
0,https://crimproject.org/data/pieces/CRIM_Mass_...,mass,CRIM_Mass_0001,Kyrie,Missa Confitemini: Kyrie,CRIM_Person_0001,Pierre Colin,1556,composer
1,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_1,NaN,Kyrie,Missa Confitemini: Kyrie,CRIM_Person_0001,Pierre Colin,1556,composer
2,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_2,NaN,Gloria,Missa Confitemini: Gloria,CRIM_Person_0001,Pierre Colin,1556,composer
3,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_3,NaN,Credo,Missa Confitemini: Credo,CRIM_Person_0001,Pierre Colin,1556,composer
4,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_4,NaN,Sanctus,Missa Confitemini: Sanctus,CRIM_Person_0001,Pierre Colin,1556,composer
5,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_5,NaN,Agnus Dei,Missa Confitemini: Agnus Dei,CRIM_Person_0001,Pierre Colin,1556,composer
6,https://crimproject.org/data/pieces/CRIM_Mass_...,mass,CRIM_Mass_0002,Kyrie,Missa Vidi speciosam: Kyrie,CRIM_Person_0003,Mathieu Sohier,1556,composer
7,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0002_1,NaN,Kyrie,Missa Vidi speciosam: Kyrie,CRIM_Person_0003,Mathieu Sohier,1556,composer
8,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0002_2,NaN,Gloria,Missa Vidi speciosam: Gloria,CRIM_Person_0003,Mathieu Sohier,1556,composer
9,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0002_3,NaN,Credo,Missa Vidi speciosam: Credo,CRIM_Person_0003,Mathieu Sohier,1556,composer


In [92]:
roles_data.columns

Index(['Unnamed: 0', 'crim_id', 'mass_id', 'title', 'full_title', 'person_id',
       'person', 'date', 'role_type'],
      dtype='object')

In [93]:
# here we combine the person and role type to make one 'word' of both of them
# this makes matching easier

roles_data['role_pairs'] = roles_data['person_id'] + roles_data['role_type']
roles_data['crim_id'].fillna("mass", inplace=True)
roles_data

,Unnamed: 0,crim_id,mass_id,title,full_title,person_id,person,date,role_type,role_pairs
0,https://crimproject.org/data/pieces/CRIM_Mass_...,mass,CRIM_Mass_0001,Kyrie,Missa Confitemini: Kyrie,CRIM_Person_0001,Pierre Colin,1556,composer,CRIM_Person_0001composer
1,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_1,NaN,Kyrie,Missa Confitemini: Kyrie,CRIM_Person_0001,Pierre Colin,1556,composer,CRIM_Person_0001composer
2,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_2,NaN,Gloria,Missa Confitemini: Gloria,CRIM_Person_0001,Pierre Colin,1556,composer,CRIM_Person_0001composer
3,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_3,NaN,Credo,Missa Confitemini: Credo,CRIM_Person_0001,Pierre Colin,1556,composer,CRIM_Person_0001composer
4,https://crimproject.org/data/pieces/CRIM_Mass_...,CRIM_Mass_0001_4,NaN,Sanctus,Missa Confitemini: Sanctus,CRIM_Person_0001,Pierre Colin,1556,composer,CRIM_Person_0001composer
...,...,...,...,...,...,...,...,...,...,...
1009,NaN,CRIM_Model_0046,NaN,Alla dolc'ombra,Alla dolc'ombra,CRIM_Person_0049,NaN,2022,editor,CRIM_Person_0049editor
1010,NaN,CRIM_Model_0047,NaN,Nigra sum,Nigra sum,CRIM_Person_0070,NaN,2022,editor,CRIM_Person_0070editor
1011,NaN,CRIM_Model_0048,NaN,Sicut lilum,Sicut lilum,CRIM_Person_0031,NaN,2022,editor,CRIM_Person_0031editor
1012,NaN,CRIM_Model_0049,NaN,Entre vous filles,Entre vous filles,CRIM_Person_0071,NaN,2022,editor,CRIM_Person_0071editor


In [94]:
def get_role_metadata_from_df(sample):
    mass = sample['mass_id']
    piece = sample['crim_id']
    role_type = sample['role_type']
    date = sample['date']
    person = sample['person_id']
    remarks = ''
    if piece == "mass":
        
        role_data = {'model': 'crim.crimrole',       
         'fields': {
             'mass': mass,
         'role_type': role_type,
        'date': date,
        'person': person,
         'remarks': remarks}}
    else:
        role_data = {'model': 'crim.crimrole',       
         'fields': {'piece': piece,
         'role_type': role_type,
        'date': date,
        'person': person,
         'remarks': remarks}}

    
    return role_data

In [97]:
# test with one example

sample = roles_data.iloc[288]
get_role_metadata_from_df(sample)

{'model': 'crim.crimrole',
 'fields': {'mass': 'CRIM_Mass_0049',
  'role_type': 'composer',
  'date': '1584',
  'person': 'CRIM_Person_0072',
  'remarks': ''}}

In [79]:
# filter out the roles already in Django data and apply transformation 

new_roles = roles_data[~roles_data['role_pairs'].isin(role_list)]

new_roles_json = new_roles.apply(get_role_metadata_from_df, axis=1)

new_roles_json

968    {'model': 'crim.crimrole', 'fields': {'piece':...
dtype: object

In [58]:
with open("roles.json", "w") as write_file:
    json.dump(roles_json, write_file, indent=4)

NameError: name 'roles_json' is not defined

In [101]:
# All Steps at once
roles = requests.get('https://crimproject.org/data/roles/').json()
roles_from_json = pd.json_normalize(roles)
roles_from_json['role_pairs'] = roles_from_json['person'] + roles_from_json['role_type']
role_list = roles_from_json['role_pairs'].to_list()

role_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vRcv1DLLiUZ3e9hIvYL6c3bW6ELJ5OJHlEVsqZvk9yKaARj4mR-onmGMBo8vGo4L9nLzA_8YpjuPNhq/pub?output=csv'

roles_data = pd.read_csv(role_csv) 
roles_data['role_pairs'] = roles_data['person_id'] + roles_data['role_type']

new_roles = roles_data[~roles_data['role_pairs'].isin(role_list)]

new_roles_json = new_roles.apply(get_role_metadata_from_df, axis=1).to_list()

with open("new_roles.json", "w") as write_file:
    json.dump(new_roles_json, write_file, indent=4)

AttributeError: 'DataFrame' object has no attribute 'to_list'

### Phrases

### People


In [ ]:
people_from_json = pd.json_normalize(people)


In [ ]:
people_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR2MuAgy8ONgJLYoyovrS961eJCwMeXiq3GcNzSQTiNLjSuEbsfll1J2se6cjN9bviI3D07tiXIrgMO/pub?output=csv'

people_data = pd.read_csv(people_csv)
people_data.columns



In [ ]:
def get_person_metadata_from_df(sample):
    person_id = sample['person_id']
    name = sample['crim_full_name']
    birth_date = sample['birth_date']
    death_date = sample['death_date']
    active_date = sample['active_date']
    remarks = sample['viaf'],
    person_data = {'model': 'crim.crimperson',       
         'fields': {'person_id': person_id,
         'name_sort': name,
        'birth_date': birth_date,
        'death_date': death_date,
        'active_date': active_date,
         'remarks': remarks}}
    return person_data

In [ ]:
# run all steps

people = requests.get('https://crimproject.org/data/people/').json()

people_from_json = pd.json_normalize(people)
people_from_json.columns
#roles_from_json['role_pairs'] = roles_from_json['person'] + roles_from_json['role_type']
people_list = people_from_json['person_id'].to_list()

people_csv = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR2MuAgy8ONgJLYoyovrS961eJCwMeXiq3GcNzSQTiNLjSuEbsfll1J2se6cjN9bviI3D07tiXIrgMO/pub?output=csv'

people_data = pd.read_csv(people_csv) 


new_people = people_data[~people_data['person_id'].isin(people_list)]

new_people_json = new_people.apply(get_person_metadata_from_df, axis=1).to_list()

with open("new_people.json", "w") as write_file:
    json.dump(new_people_json, write_file, indent=4)

### Useful Code

In [ ]:
df['observer.name'].value_counts()

In [ ]:
mask = df['observer.name'].str.contains("Lorenz")
lorenz_list = df[mask]
lorenz_list['piece.full_title'].value_counts()
# fiala_list = df[mask]
# fiala_list['piece.full_title'].value_counts()

In [ ]:
curated_obs['piece_id'].str.contains("Model_0013")


In [ ]:
df["measures"] = df['ema'].str.extract('(\d+-\d+)')